In [1]:
import os
import os.path
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt

import skimage.io as sio

/users/visics/kkelchte/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Define which folders to parse:
root_dir='/esat/opal/kkelchte/docker_home/pilot_data/almost_collision_set'
# root_dir='/esat/opal/kkelchte/docker_home/pilot_data/small'
runs=sorted([root_dir+'/'+d for d in os.listdir(root_dir) if ('garage' in d) or ('corridor' in d) or ('caffeteria'in d) or ( 'office' in d) or ('outside' in d) ])
print len(runs),' runs.'
print [os.path.basename(r) for r in runs]

25  runs.
['caffeteria_00000_strange_left', 'caffeteria_00001_strange_right', 'caffeteria_00002_strange_right', 'caffeteria_00003_strange_left', 'corridor_down_00000_perspective_left', 'corridor_down_00000_perspective_right', 'corridor_telemic_00000_perspective_right', 'corridor_telemic_00001_perspective_left', 'corridor_telemic_00004_vertical_straight', 'corridor_telemic_00005_perspective_right', 'corridor_telemic_00006_perspective_left', 'garage_00000_perspective_left', 'garage_00000_perspective_right', 'garage_00000_vertical_left', 'garage_00000_vertical_right', 'garage_corridor_00001_perspective_right', 'garage_corridor_00002_perspective_left', 'office_00000_strange_left', 'office_00000_strange_right', 'outside_night_00000_perspective_right', 'outside_night_00000_strange_left', 'outside_night_00000_strange_right', 'outside_night_00000_vertical_left', 'outside_night_00000_vertical_right', 'outside_night_00001_perspective_left']


In [4]:
root_name= 'data.hdf5'
# remove if it already exists
if os.path.isfile(root_dir+'/'+root_name):
    os.remove(root_dir+'/'+root_name)
f = h5py.File(root_dir+'/'+root_name, 'w')

In [5]:
# create a group for this run
for r in runs: f.create_group(os.path.basename(r))

In [6]:
def add_dir(group, data_type='RGB'):
    if os.path.isdir(r+'/'+data_type):
        # get an idea of the shape
        image_files=sorted([r+'/'+data_type+'/'+i for i in os.listdir(r+'/'+data_type)])
        img=sio.imread(image_files[0])
        try:
            img_array=np.zeros((len(image_files), img.shape[0], img.shape[1], img.shape[2]),dtype=np.uint8)
        except:
            img_array=np.zeros((len(image_files), img.shape[0], img.shape[1]),dtype=np.uint8)
        for i, imgf in enumerate(image_files): img_array[i]=sio.imread(imgf)
#         plt.imshow(img_array[-1])
#         plt.show()
        print('extracted {0}, shape: {1}'.format(data_type,img_array.shape))
        return group.create_dataset(data_type,data=img_array, compression="gzip", compression_opts=8)
#         return group.create_dataset(data_type,data=img_array, compression="lzf")
    else:
        print('could not find {0} in {1}.'.format(data_type,group.name))
        return []


In [7]:
def add_file(group, file_name='control_info'):
    if os.path.isfile(r+'/'+file_name+'.txt'):
        info_file=open(r+'/'+file_name+'.txt','r')
        info=np.asarray([[float(i) for i in l.split(' ')]  for l in info_file.readlines()])
        print('extracted {0}, shape: {1}'.format(file_name, info.shape))
#         return group.create_dataset(file_name,data=info, compression="gzip", compression_opts=9)
        return group.create_dataset(file_name,data=info, compression="gzip", compression_opts=2)
    else:
        print('could not find {0} in {1}.'.format(file_name,group.name))
        return []

In [8]:
while [ len(runs) != 0 ]:
    r=runs.pop(0)
    print("[{1[2]}/{1[1]}_{1[3]:02d}:{1[4]:02d}]: {0} left".format(len(runs),tuple(time.localtime()[0:5])))
    add_dir(f[os.path.basename(r)],'RGB')
    add_dir(f[os.path.basename(r)],'Depth')
    add_dir(f[os.path.basename(r)],'Depth_predicted')
    add_file(f[os.path.basename(r)],'control_info')
    add_file(f[os.path.basename(r)],'collision_info')
    add_file(f[os.path.basename(r)],'collision_info_3')
    add_file(f[os.path.basename(r)],'collision_info_5')
    add_file(f[os.path.basename(r)],'collision_info_7')
    f.flush()

[16/4_17:05]: 24 left
extracted RGB, shape: (52, 480, 856, 3)
could not find Depth in /caffeteria_00000_strange_left.
could not find Depth_predicted in /caffeteria_00000_strange_left.
extracted control_info, shape: (63, 7)
could not find collision_info in /caffeteria_00000_strange_left.
could not find collision_info_3 in /caffeteria_00000_strange_left.
could not find collision_info_5 in /caffeteria_00000_strange_left.
could not find collision_info_7 in /caffeteria_00000_strange_left.
[16/4_17:05]: 23 left
extracted RGB, shape: (52, 480, 856, 3)
could not find Depth in /caffeteria_00001_strange_right.
could not find Depth_predicted in /caffeteria_00001_strange_right.
extracted control_info, shape: (52, 7)
could not find collision_info in /caffeteria_00001_strange_right.
could not find collision_info_3 in /caffeteria_00001_strange_right.
could not find collision_info_5 in /caffeteria_00001_strange_right.
could not find collision_info_7 in /caffeteria_00001_strange_right.
[16/4_17:05]: 22

extracted RGB, shape: (69, 480, 856, 3)
could not find Depth in /garage_corridor_00002_perspective_left.
could not find Depth_predicted in /garage_corridor_00002_perspective_left.
extracted control_info, shape: (76, 7)
could not find collision_info in /garage_corridor_00002_perspective_left.
could not find collision_info_3 in /garage_corridor_00002_perspective_left.
could not find collision_info_5 in /garage_corridor_00002_perspective_left.
could not find collision_info_7 in /garage_corridor_00002_perspective_left.
[16/4_17:07]: 7 left
extracted RGB, shape: (51, 480, 856, 3)
could not find Depth in /office_00000_strange_left.
could not find Depth_predicted in /office_00000_strange_left.
extracted control_info, shape: (70, 7)
could not find collision_info in /office_00000_strange_left.
could not find collision_info_3 in /office_00000_strange_left.
could not find collision_info_5 in /office_00000_strange_left.
could not find collision_info_7 in /office_00000_strange_left.
[16/4_17:07]: 6

IndexError: pop from empty list

In [71]:
# Request that the HDF5 library flush its buffers to disk.
f.flush()

In [75]:
# Read in hdf5 file to check if everything is there:
f = h5py.File(root_dir+'/data.hdf5', 'r')
print 'Reading data.hdf5'
for g in f.keys():
    print g
    for ds in f[g].keys():
        print '---{0}: {1}'.format(ds, f[g][ds].shape)
f.close()

Reading data.hdf5
00000_canyon
---Depth: (130, 360, 640)
---Depth_predicted: (119, 128, 160)
---RGB: (129, 360, 640, 3)
---collision_info: (129, 2)
---collision_info_3: (129, 2)
---collision_info_5: (129, 2)
---collision_info_7: (129, 2)
---control_info: (258, 7)
00001_canyon
---Depth: (96, 360, 640)
---Depth_predicted: (95, 128, 160)
---RGB: (95, 360, 640, 3)
---collision_info: (95, 2)
---collision_info_3: (95, 2)
---collision_info_5: (95, 2)
---collision_info_7: (95, 2)
---control_info: (191, 7)
